Identificación de créditos riesgosos
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/machine-learning/blob/master/01-archivos-y-directorios.ipynb) para acceder a la última versión online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/machine-learning/blob/master/01-archivos-y-directorios.ipynb) para ver la última versión online en `nbviewer`. 

---
[Licencia](https://github.com/jdvelasq/machine-learning/blob/master/LICENCIA.txt)  
[Readme](https://github.com/jdvelasq/machine-learning/blob/master/readme.md)

# Definición del problema real

En el tutorial anterior se construyó un clasificador que tiene como objetivo reprobar o aprobar solicitudes de crédito. El gerente de proyecto ha solitado que se haga un análisis del desempeño del sistema cuando se encuentre en productivo.

# Definición del problema en términos de los datos

El problema radica en que los datos para entrenamiento y prueba fueron tomados de forma determinística, pero los resultados del desempeño del clasificador podrían variar si la partición de los datos se hace diferente. Más aún, en vez de realizar estimaciones puntuales de la métrica de error seleccionada, sería mucho más conveniente estimar su distribución de probabilidades. 

In [5]:
# se carga la librería
library(C50)

# lectura de los datos
credit <- read.csv("data/credit.csv")
credit$default <- factor(credit$default, labels=c("No", "Yes"))

### Solución 1.

In [20]:
# En esta solución se realiza una partición aleatoria
# de los datos en los conjuntos de entrenamiento y prueba
set.seed(123456)   # se reinicia la semilla del generador aleatorio
train_sample <- sample(1000, 900)
str(train_sample)

 int [1:900] 798 753 391 341 360 198 532 96 980 167 ...


In [18]:
credit_train <- credit[train_sample, ]
credit_test  <- credit[-train_sample, ]
credit_model <- C5.0(credit_train[-17], credit_train$default)
credit_pred <- predict(credit_model, credit_test)
length(credit_test$default[credit_pred != credit_test$default])

[1] 30

In [25]:
# install.packages('irr')
library(irr)
library(caret)
folds <- createFolds(credit$default, k = 10)
str(folds)

also installing the dependency ‘lpSolve’




The downloaded binary packages are in
	/var/folders/yq/svn60mh123z6dzr3d4rjk_740000gn/T//Rtmpykflrs/downloaded_packages


Loading required package: lpSolve


List of 10
 $ Fold01: int [1:100] 23 51 85 91 92 97 101 106 125 136 ...
 $ Fold02: int [1:100] 9 20 26 27 37 41 58 68 76 82 ...
 $ Fold03: int [1:100] 2 5 6 8 11 13 19 32 34 40 ...
 $ Fold04: int [1:100] 4 10 28 48 78 83 120 132 134 142 ...
 $ Fold05: int [1:100] 21 63 74 87 103 107 108 112 119 121 ...
 $ Fold06: int [1:100] 22 29 31 36 39 42 45 57 59 64 ...
 $ Fold07: int [1:100] 12 14 17 33 35 43 44 52 62 65 ...
 $ Fold08: int [1:100] 7 15 16 18 25 46 67 72 84 86 ...
 $ Fold09: int [1:100] 3 49 50 53 56 60 70 71 98 102 ...
 $ Fold10: int [1:100] 1 24 30 38 47 61 81 105 109 116 ...


In [26]:
cv_results <- lapply(folds, function(x) {
    credit_train <- credit[-x, ]
    credit_test <- credit[x, ]
    credit_model <- C5.0(default ~ ., data = credit_train)
    credit_pred <- predict(credit_model, credit_test)
    credit_actual <- credit_test$default
    kappa <- kappa2(data.frame(credit_actual, credit_pred))$value
    return(kappa)
  })
str(cv_results)

List of 10
 $ Fold01: num 0.266
 $ Fold02: num 0.475
 $ Fold03: num 0.255
 $ Fold04: num 0.175
 $ Fold05: num 0.255
 $ Fold06: num 0.193
 $ Fold07: num 0.417
 $ Fold08: num 0.381
 $ Fold09: num 0.303
 $ Fold10: num 0.475


In [28]:
mean(unlist(cv_results))

[1] 0.3195432